In [1]:
from openai import OpenAI

class PatientSymptomDescr:

    def __init__(self):
        self.gender = 'female'
        self.age = 2
        self.weight = 33
        self.bmi = 22
        self.somatic_symptoms = ['fatigue', 'extreme weakness']
        self.physical_exam_findings = ['rash in face']
        self.other_symptoms = ['frequent colds', 'occassional fever']

    def make_prompt(self):
        s = f"""
        The patient is {self.age} year old {self.gender} who weights {self.weight} pounds with a BMI of {self.bmi}. 
        {self.get_symptoms_string()}
        """
        return s
    
    def make_item_list(self, lst):
        if len(lst) > 1:
            return ','.join(lst[:-1]) + ' and ' + lst[-1]
        else:
            return lst[-1]
    
    def get_symptoms_string(self):
        somatic = ('The patient\'s complaints include ' + self.make_item_list(self.somatic_symptoms)+'. ') if len(self.somatic_symptoms) > 0 else ''
        physical = ('Examining the patient, I found ' + self.make_item_list(self.physical_exam_findings) +'. ') if len(self.physical_exam_findings) > 0 else ''
        other = ('Other symptoms are ' + self.make_item_list(self.other_symptoms) +'. ') if len(self.other_symptoms) > 0 else ''
        return somatic + physical +  other 
    


def prompt_open_ai(prompt_str, model="gpt-3.5-turbo"):

    client = OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=[
        {
        "role": "system",
        "content": """ I am taking care of a patient and would like your help generating a differential diagnosis.  I will provide details 
        about the patient and the symptoms below. Please output upto 5 conditions that are consistent with the symptoms sorted from the most likely to the least. Your 
        output must be in the format wherein each of the conditions must be in a separate line.
                        1. <name of condition/disease>, <probability of this disease as a percentage>, <brief rationale as to why the patient may have this disease>
                        2. <name of condition/disease>, <probability of this disease as a percentage>, <brief rationale as to why the patient may have this disease>
                        ...
                        5. <name of condition/disease>, <probability of this disease as a percentage>, <brief rationale as to why the patient may have this disease>
                        """
        },
        {
        "role": "user",
        "content": prompt_str
        }
        ],
        temperature=0.7,
        max_tokens=1024,
        top_p=1
    )
    print(response)
    print(response.choices[0].message.content)
    return response.choices[0].message.content

In [4]:
p = PatientSymptomDescr()
print(p.get_symptoms_string())
print(p.make_prompt())
msg = prompt_open_ai(p.make_prompt())

The patient's complaints include fatigue and extreme weakness. Examining the patient, I found rash in face. Other symptoms are frequent colds and occassional fever. 

        The patient is 2 year old female who weights 33 pounds with a BMI of 22. 
        The patient's complaints include fatigue and extreme weakness. Examining the patient, I found rash in face. Other symptoms are frequent colds and occassional fever. 
        
ChatCompletion(id='chatcmpl-9HfvOXcDEb4nvGxzfEhXO01HQGNqL', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="1. Juvenile Idiopathic Arthritis, 40%, The patient's rash on the face could be indicative of a rheumatologic condition like Juvenile Idiopathic Arthritis, which can present with fatigue, weakness, and occasional fever in children.\n2. Systemic Lupus Erythematosus (SLE), 25%, SLE can present with fatigue, weakness, rash on the face, and frequent colds in children. It is important to consider autoimmune co

In [17]:
parse(msg)

1. Juvenile Idiopathic Arthritis. 40%. The patient's rash on the face could be indicative of a rheumatologic condition like Juvenile Idiopathic Arthritis.
2. Systemic Lupus Erythematosus (SLE). 25%. SLE can present with fatigue.
3. Iron Deficiency Anemia. 15%. Fatigue and weakness are common symptoms of anemia.
4. Viral Infection. 10%. The patient's symptoms of fatigue.
5. Chronic Fatigue Syndrome. 10%. Chronic Fatigue Syndrome can present with fatigue.


[('Juvenile Idiopathic Arthritis',
  40,
  "The patient's rash on the face could be indicative of a rheumatologic condition like Juvenile Idiopathic Arthritis"),
 ('Systemic Lupus Erythematosus (SLE)', 25, 'SLE can present with fatigue'),
 ('Iron Deficiency Anemia',
  15,
  'Fatigue and weakness are common symptoms of anemia'),
 ('Viral Infection', 10, "The patient's symptoms of fatigue"),
 ('Chronic Fatigue Syndrome',
  10,
  'Chronic Fatigue Syndrome can present with fatigue')]